Demonstrations for the theory of <a class="ProveItLink" href="theory.ipynb">proveit.core_expr_types.tuples</a>
========

In [ ]:
import proveit
from proveit import (Lambda, ExprTuple, ExprRange, IndexedVar, safe_dummy_var,
                     defaults, Function, ProofFailure)
from proveit import A, a, b, f, g, i, j, k, l, m, n, x, y
from proveit.logic import Equals, NotEquals, EvaluationError, InSet
from proveit.linear_algebra import TensorProd
from proveit.numbers import (zero, one, two, three, four, e as e_nat, Add, Exp,
                             Integer, Mult, Natural, NaturalPos, Neg, num, subtract)
from proveit.core_expr_types import Len
from proveit.core_expr_types import (
        a_i_to_j, x_1_to_n, y_1_to_n, f_i_to_j, f_i_to_jp1)
from proveit.core_expr_types.tuples import (
        len_of_ranges_with_repeated_indices_from_1, range_from1_len,
        len_of_ranges_with_repeated_indices_from_1, len_of_ranges_with_repeated_indices,
        len_of_empty_range_of_ranges)
from proveit.core_expr_types.tuples import general_range_of_ranges
%begin demonstrations

### Proving the equality of ExprTuples

In [ ]:
Equals(ExprTuple(a, b, x, y), ExprTuple(a, b, x, y)).prove()

In [ ]:
NotEquals(ExprTuple(a, b, x, y), ExprTuple(x, y)).prove()

In [ ]:
NotEquals(ExprTuple(a, b, x, y), ExprTuple(x, b, x, y)).prove(assumptions=[NotEquals(a, x)])

In [ ]:
partitioned = x_1_to_n.partition(one, assumptions=[InSet(n, NaturalPos)])

In [ ]:
eq = Equals(ExprTuple(*partitioned.lhs.entries, y), 
            ExprTuple(*partitioned.rhs.entries, y))

In [ ]:
eq.lhs.inner_expr(assumptions=[InSet(n, NaturalPos)])[0].partition(one, assumptions=[InSet(n, NaturalPos)])

In [ ]:
eq.lhs.deduce_equality(eq, assumptions=[InSet(n, NaturalPos)])

In [ ]:
Equals(ExprTuple(*partitioned.lhs.entries, y), 
       ExprTuple(*partitioned.rhs.entries, y)).prove(
    assumptions=[InSet(n, NaturalPos)])

### Testing the `ExprRange(order='decreasing')` flag

An ExprRange with the `order='decreasing'` flag produces an ExprRange of the form f(i),...,f(j), where 0 <= (i-j) and j < i (i.e. the ExprRange represents 0 or more elements and is in decreasing order)

For example, if

self = $f(3),...,f(6)$,

then `order='decreasing'` would return:

$|- f(6),...,f(3)$

In [ ]:
test = ExprRange(i, IndexedVar(a, i), j, zero, order='decreasing')

In [ ]:
inc_test = ExprRange(i, IndexedVar(a, i), j, zero)

In [ ]:
try:
    ExprRange(i, IndexedVar(a, i), two, zero, order='increasing')
    assert False, "Expecting a ValueError; should not get this far!"
except ValueError as _e:
    print("Expected exception: ", _e)

In [ ]:
ExprRange(i, IndexedVar(a, i), two, zero, order='decreasing')

In [ ]:
test.lambda_map

In [ ]:
(test.start_index, test.end_index, test.true_start_index, test.true_end_index)

In [ ]:
(inc_test.start_index, inc_test.end_index, 
 inc_test.true_start_index, inc_test.true_end_index)

In [ ]:
test.body

In [ ]:
inc_test.body

In [ ]:
test.first()

In [ ]:
inc_test.first()

In [ ]:
test.last()

In [ ]:
inc_test.last()

In [ ]:
empty_test_inc = ExprRange(i, IndexedVar(a, i), one, zero, order='increasing')

In [ ]:
empty_test_inc.reduction()

In [ ]:
empty_test_dec = ExprRange(i, IndexedVar(a, i), zero, one, order='decreasing')

In [ ]:
empty_test_dec.reduction()

In [ ]:
empty_test_dec.lambda_map

In [ ]:
partition_test_inc = ExprRange(i, IndexedVar(a, i), zero, four, order='increasing')

In [ ]:
partition_test_inc.partition(two)

In [ ]:
partition_test_dec = ExprRange(i, IndexedVar(a, i), four, zero, order='decreasing')

In [ ]:
partition_test_dec.start_index, partition_test_dec.true_start_index

In [ ]:
partition_test_dec.partition(two)

In [ ]:
from proveit import fa
from proveit.numbers import Complex

In [ ]:
assumptions = [InSet(Add(j, Neg(i), one), Natural), 
               InSet(i, Natural), InSet(j, Natural)]

In [ ]:
f_i_to_j_dec = ExprRange(a, fa, i, j, order='decreasing')

In [ ]:
from proveit.core_expr_types.tuples import negated_shift_equivalence

In [ ]:
negated_shift_equivalence

In [ ]:
# negated_shift_equivalence.instantiate({i:Neg(i), j:Neg(j), k:subtract(i, one), a:Neg(one), l:subtract(j, one)}, assumptions=[InSet(subtract(subtract(Neg(j), one), Neg(i)),
# #                                                                  Natural), InSet(subtract(subtract(j, one), i),
#                                                                  Natural)])

In [ ]:
# f_i_to_j_dec.shift_equivalence(new_shift=one, assumptions=[InSet(subtract(subtract(Neg(j), one), Neg(i)),
#                                                                  Natural)])

In [ ]:
# f_i_to_j__shift__eq_dec = \
#     f_i_to_j_dec.shift_equivalence(new_shift=one, assumptions=[InSet(subtract(subtract(Neg(j), one), Neg(i)),
#                                                                  Natural)]).lhs.entries[0]

In [ ]:
# assert False

In [ ]:
Len(ExprRange(i, ExprRange(j, IndexedVar(a, [i, j]), one, zero), one, n)).computation(assumptions=[InSet(n, NaturalPos)])

In [ ]:
Len(ExprRange(i, ExprRange(j, IndexedVar(a, [i, j]), two, n), one, zero)).computation(assumptions=[InSet(n, Natural)])

In [ ]:
Len(ExprRange(i, ExprRange(j, IndexedVar(a, [i, j]), one, n), one, n)).computation(assumptions=[InSet(n, Natural), InSet(n, NaturalPos)])

In [ ]:
Len(ExprRange(i, ExprRange(j, IndexedVar(a, [i, j]), one, zero), two, one)).computation(assumptions=[InSet(n, Natural)])

The following 5 cells have been commented out — they create and then depend on the one-element expr_range in the first cell, which returns an error

In [ ]:
# expr_range = ExprRange(i, ExprRange(j, IndexedVar(a, [i, j]), one, one), one, one)

In [ ]:
# lambda_map = Lambda((expr_range.parameter, expr_range.body.parameter), expr_range.body.body)

In [ ]:
# singular_nested_range_reduction.instantiate({f:lambda_map, i:one, j:one, m:one})

In [ ]:
# Len(ExprRange(i, ExprRange(j, IndexedVar(a, [i, j]), one, one), one, one)).computation()

In [ ]:
# Len(ExprRange(i, ExprRange(j, IndexedVar(a, [i, j]), one, two), one, one)).computation()

In [ ]:
assumptions = [InSet(Add(j, Neg(i), one), Natural), 
               InSet(i, Natural), InSet(j, Natural)]

In [ ]:
f_i_to_jp1

In [ ]:
eq = f_i_to_jp1.partition(j, assumptions=assumptions)

In [ ]:
eq.rhs.merger(assumptions=assumptions)

In [ ]:
f_i_to_j__shift__eq = \
    f_i_to_j.shift_equivalence(new_shift=one, assumptions=[InSet(subtract(Add(j, one), i),
                                                                 Natural)])

In [ ]:
f_i_to_j__shift__eq.rhs

In [ ]:
Add(subtract(i, one), one).simplification(assumptions=[InSet(i, Integer)])

In [ ]:
Add(Add(i, Neg(two)), two).simplification(assumptions=[InSet(i, Integer)])

In [ ]:
Add(subtract(j, one), one).simplification(assumptions=[InSet(j, Integer)])

In [ ]:
Add(subtract(j, two), two).simplification(assumptions=[InSet(j, Integer)])

In [ ]:
f_i_to_j__shift__eq.rhs.entries[0].shift_equivalence(
    old_shift=one, new_shift=two, 
    assumptions=[InSet(j, Integer), InSet(i, Integer),
                 InSet(subtract(j, subtract(i, one)), Natural)])

In [ ]:
from proveit import fa
from proveit.numbers import Complex

In [ ]:
f_i_to_j_dec = ExprRange(a, fa, i, j, order='decreasing')

In [ ]:
assumptions = [InSet(Add(Neg(j), i, Neg(one)), Natural), 
               InSet(i, Natural), InSet(j, Natural)]

In [ ]:
#InSet(Neg(Add(x, one)), Complex).prove(assumptions=[InSet(x, Complex)])

In [ ]:
Neg(Add(Neg(Add(x, one)), one)).distribution(assumptions=[InSet(x, Complex)])

In [ ]:
InSet(subtract(subtract(Neg(j), one), Neg(i)), Natural)

In [ ]:
# f_i_to_j__shift__eq_dec = \
#     f_i_to_j_dec.shift_equivalence(new_shift=one, assumptions=[InSet(subtract(subtract(Neg(j), one), Neg(i)),
#                                                                  Natural)]).lhs.entries[0]

In [ ]:
# lhs_done = f_i_to_j__shift__eq_dec.inner_expr().lhs[0].with_decreasing_order()

In [ ]:
# lhs_done.inner_expr().rhs[0].with_decreasing_order()

In [ ]:
# f_i_to_j__shift__eq_dec.rhs.entries[0].shift_equivalence(
#     old_shift=one, new_shift=two, 
#     assumptions=[InSet(j, Integer), InSet(i, Integer),
#                  InSet(subtract(j, subtract(i, one)), Natural)])

In [ ]:
tup=ExprTuple(a, b)

In [ ]:
Len(a_i_to_j).computation(assumptions=[InSet(Add(j, Neg(i), one), Natural)])

In [ ]:
tup_jdgmt = tup.prove(assumptions=[tup])

Not clear what the following cell is/was trying to accomplish:

In [ ]:
# tup.prove(assumptions=[tup]).instantiate({a:a_i_to_j},
#                             assumptions=[InSet(Add(j, Neg(i), one), Natural)])

In [ ]:
eq = f_i_to_jp1.partition(k, assumptions=[InSet(subtract(Add(k, one), i), Natural),
                              InSet(subtract(Add(j, one), k), Natural)])

In [ ]:
eq.rhs.merger(assumptions=[InSet(subtract(Add(k, one), i), Natural),
                              InSet(subtract(Add(j, one), k), Natural)])

In [ ]:
eq = f_i_to_jp1.partition(i, assumptions=[InSet(subtract(Add(j, one), i), Natural)])

In [ ]:
eq.rhs.merger(assumptions=[InSet(subtract(Add(j, one), i), Natural)])

In [ ]:
eq = f_i_to_j.partition(subtract(j, one), assumptions=[InSet(subtract(j, i), Natural)])

In [ ]:
Add(subtract(j, one), one).simplification(assumptions=[InSet(j, Natural)])

In [ ]:
eq.rhs.merger(assumptions=[InSet(subtract(j, i), Natural), InSet(j, Natural)])

### Testing the `ExprTuple.range_expansion()` method

For self an ExprTuple with a single entry that is an ExprRange of the form f(i),...,f(j), where 0 <= (j-i) <= 9 (i.e. the ExprRange represents 1 to 10 elements), derive and return an equality between self and an ExprTuple with explicit entries replacing the ExprRange. For example, if

self = ExprTuple$(f(3),...,f(6))$,

then `self.range_expansion()` would return:

$|- (f(3),...,f(6)) = (f(3), f(4), f(5), f(6))$

First not that we cannot create an ExprRange with only one element explicitly:

In [ ]:
try:
    ExprTuple(ExprRange(a, Function(g, a), two, two))
    assert False, "Expecting a ValueError"
except ValueError as e:
    print("Expected Error:", e)

We can, however, create an ExprRange that doesn't obviously have one element but may turn out to have one element after a simplification or substitution.

In [ ]:
expr_tuple_with_1_elem_range = ExprTuple(ExprRange(a, Function(g, a), two, Mult(one, two)))

In [ ]:
expr_tuple_with_1_elem_range.simplification()

ExprRange indices are already expected to be simplified w.r.t. `quick_simplified_index` from `proveit.numbers`, however, which deals with basic `Add` and `Neg` simplifications and combines all literal integers at the end of an `Add`.

In [ ]:
try:
    # A one-element ExprRange (hidden by choosing the start and end indices
    # that are equal but appear different)
    example_expr_range_2 = ExprRange(i, IndexedVar(x, i), one, Add(one, zero))
    assert False, "Expecting ValueError"
except ValueError as e:
    print("Expected Error:", e)

`range_expansion` does the same thing as `simplification` in this case:

In [ ]:
expr_tuple_with_1_elem_range.range_expansion()

In [ ]:
# generate some example ExprTuples each containing a single ExprRange element
expr_tuple_with_2_elem_range, expr_tuple_with_2_elem_range_exp, expr_tuple_with_3_elem_range, expr_tuple_with_unknown_elem_range, expr_tuple_too_big = [
        ExprTuple(ExprRange(a, Function(g, a), two, three)),
        ExprTuple(ExprRange(a, Exp(two, Add(a, one)), two, three)),
        ExprTuple(ExprRange(a, Exp(e_nat, Add(a, j)), two, four)),
        ExprTuple(ExprRange(a, Exp(e_nat, Add(a, j)), i, k)),
        ExprTuple(ExprRange(a, Function(g, a), two, num(20)))]

In [ ]:
# find equivalent explicitly 2-element ExprTuple
expr_tuple_with_2_elem_range.range_expansion()

In [ ]:
# find equivalent explicitly 2-element ExprTuple
# notice the automatic simplification
expr_tuple_with_2_elem_range_exp.range_expansion()

In [ ]:
# find equivalent explicitly 2-element ExprTuple
# notice the automatic simplification
expr_tuple_with_3_elem_range.range_expansion()

In [ ]:
# When the number of elements represented by the ExprRange is not known,
# the range_expansion() method fails with an informative error message
try:
    expr_tuple_with_unknown_elem_range.range_expansion()
    assert False, "Expecting an EvaluationError; should not get this far!"
except EvaluationError as the_error:
    print("EvaluationError: {}".format(the_error))

In [ ]:
# but if we give the method enough information about the indexing
# we can generate the equivalent explicit ExprTuple
expr_tuple_with_unknown_elem_range.range_expansion(
        assumptions=[Equals(k, Add(i, four)), Equals(subtract(k, i), four)])

In [ ]:
# When the number of elements represented by the ExprRange is > 10,
# the range_expansion() method fails with an informative error message
try:
    expr_tuple_too_big.range_expansion(assumptions=[Equals(subtract(num(20), two), num(18))])
    assert False, "Expecting an EvaluationError; should not get this far!"
except ValueError as the_error:
    print("ValueError: {}".format(the_error))

In [ ]:
# a related issue to ask about
try:
    Equals(subtract(num(20), two), num(18)).prove()
except ProofFailure as the_error:
    print("ProofFailure: {}".format(the_error))

A related example where the ExprRange and ExprTuple are being used as an argument for a TensorProd operator.

In [ ]:
# create an example ExprRange
example_expr_range = ExprRange(i, IndexedVar(x, i), one, three)

In [ ]:
# use the example ExprRange as the arg(s) for a TensorProd:
example_tensor_prod_over_range = TensorProd(example_expr_range)

In [ ]:
# find the ExprTuple equivalent to the (ExprTuple-wrapped) ExprRange:
expr_range_eq_expr_tuple = ExprTuple(example_expr_range).range_expansion()

In [ ]:
# find the equivalent explicit tensor prod
example_tensor_prod_over_range.inner_expr().operands.substitution(
         expr_range_eq_expr_tuple)

In [ ]:
%end demonstrations